# Import des modules


In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import Counter
from sklearn.pipeline import Pipeline

#Selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold, StratifiedKFold

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance

from sklearn.preprocessing import label_binarize

#Modèles
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#Metriques
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_recall_curve, average_precision_score, roc_curve, auc

In [124]:
fc = pd.read_csv('fc_after_feature_engineering.csv')
print(fc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 40 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   a_quitte_l_entreprise                      1470 non-null   bool   
 1   age                                        1470 non-null   int64  
 2   annees_dans_l_entreprise                   1470 non-null   int64  
 3   annees_dans_le_poste_actuel                1470 non-null   int64  
 4   annees_depuis_la_derniere_promotion        1470 non-null   int64  
 5   annees_experience_totale                   1470 non-null   int64  
 6   annes_sous_responsable_actuel              1470 non-null   int64  
 7   augmentation_salaire_precedente            1470 non-null   int64  
 8   distance_domicile_travail                  1470 non-null   int64  
 9   domaine_etude_Entrepreunariat              1470 non-null   float64
 10  domaine_etude_Infra & Cl

# Déclaration des Dataframes X et y
- Un DataFrame contenant les features => X
- Un Pandas Series contenant la colonne cible => y

In [125]:
#Un homme , Célibataire , Entre 30 et 40 ans , Consultant , Entre 1 et 7 années dans l’entreprise, Un revenu compris entre 2500€ et 6000€, Domaine d’etude Infra & Cloud, Qui a moins de 5 années sous son responsable actuel, Qui a une distance domicile travail entre 3 et 17km

columns_base = ['genre', 'statut_marital', 'age', 'annees_dans_l_entreprise', 'revenu_mensuel', 'distance_domicile_travail', 'satisfaction_globale']

columns_domaine_etude = [col for col in fc.columns if col.startswith('domaine_etude')]
columns_poste = [col for col in fc.columns if col.startswith('poste')]

all_columns = columns_base + columns_domaine_etude + columns_poste

X = fc[all_columns]
y = fc['a_quitte_l_entreprise']

#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=666)

# Fonction de Séparation Train Test
- Des métriques d’évaluation calculées pour chaque modèle, sur le jeu d’apprentissage et le jeu de test.

In [126]:
def train_test_evaluation(model, X_train, X_test, y_train, y_test, model_name):
    """
    Entraîne un modèle et retourne les métriques train/test avec détection d'overfitting.

    Args:
        model: Le modèle à entraîner
        X_train, X_test: Features d'entraînement et de test
        y_train, y_test: Labels d'entraînement et de test
        model_name: Nom du modèle (string)

    Returns:
        dict: Dictionnaire contenant les résultats avec métriques train et test
    """
    # Entraînement
    model.fit(X_train, y_train)

    # Prédictions sur TRAIN
    y_pred_train = model.predict(X_train)
    report_train = classification_report(y_train, y_pred_train, output_dict=True)

    # Prédictions sur TEST
    y_pred_test = model.predict(X_test)
    report_test = classification_report(y_test, y_pred_test, output_dict=True)

    # Matrice de confusion (test)
    cm = confusion_matrix(y_test, y_pred_test)

    # Calcul des écarts (train - test) pour détecter l'overfitting
    accuracy_gap = report_train['accuracy'] - report_test['accuracy']
    f1_gap = report_train['macro avg']['f1-score'] - report_test['macro avg']['f1-score']

    # Indicateur d'overfitting (seuil à 5% d'écart)
    overfitting_flag = 'OUI' if (accuracy_gap > 0.12 or f1_gap > 0.12) else 'NON'

    # Extraction des métriques principales
    results = {
        'model': model_name,
        'method': 'train_test',
        # Métriques TRAIN
        'train_accuracy': report_train['accuracy'],
        'train_f1_macro': report_train['macro avg']['f1-score'],
        'train_precision_macro': report_train['macro avg']['precision'],
        'train_recall_macro': report_train['macro avg']['recall'],
        # Métriques TEST
        'test_accuracy': report_test['accuracy'],
        'test_f1_macro': report_test['macro avg']['f1-score'],
        'test_precision_macro': report_test['macro avg']['precision'],
        'test_recall_macro': report_test['macro avg']['recall'],
        # Écarts et overfitting
        'accuracy_gap': accuracy_gap,
        'f1_gap': f1_gap,
        'overfitting': overfitting_flag,
        'confusion_matrix': str(cm.tolist()),
    }

    # Vérifier si le modèle supporte predict_proba
    try:
        y_pred_proba = model.predict_proba(X_test)
    except AttributeError:
        y_pred_proba = None  # Pour les modèles sans probabilités

    curves = {
        'model': model_name,
        'y_true': y_test.copy() if hasattr(y_test, 'copy') else y_test,
        'y_pred_proba': y_pred_proba,
        'y_pred': y_pred_test
    }

    return results, report_test, cm, curves

# Fonction de Validation Croisée
- cross_validate

In [127]:
def cross_validation_evaluation(model, X, y, model_name, cv=5):
    """
    Effectue une validation croisée et retourne les métriques avec détection d'overfitting.

    Args:
        model: Le modèle à évaluer
        X: Features complètes
        y: Labels complets
        model_name: Nom du modèle (string)
        cv: Nombre de folds (défaut: 5)

    Returns:
        dict: Dictionnaire contenant les résultats moyens avec scores train et test
    """
    # Définition des métriques à calculer
    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

    # Validation croisée avec return_train_score=True pour détecter overfitting
    cv_results = cross_validate(model, X, y, cv=cv, scoring=scoring,
                                return_train_score=True)

    # Calcul des écarts moyens (train - test)
    accuracy_gap_cv = cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean()
    f1_gap_cv = cv_results['train_f1_macro'].mean() - cv_results['test_f1_macro'].mean()

    # Indicateur d'overfitting
    overfitting_flag_cv = 'OUI' if (accuracy_gap_cv >= 0.12 or f1_gap_cv >= 0.12) else 'NON'

    # Calcul des moyennes et écarts-types
    results = {
        'model': model_name,
        'method': 'cross_validation',
        # Métriques TRAIN
        'train_accuracy': cv_results['train_accuracy'].mean(),
        'train_accuracy_std': cv_results['train_accuracy'].std(),
        'train_f1_macro': cv_results['train_f1_macro'].mean(),
        'train_f1_macro_std': cv_results['train_f1_macro'].std(),
        'train_precision_macro': cv_results['train_precision_macro'].mean(),
        'train_recall_macro': cv_results['train_recall_macro'].mean(),
        # Métriques TEST
        'test_accuracy': cv_results['test_accuracy'].mean(),
        'test_accuracy_std': cv_results['test_accuracy'].std(),
        'test_f1_macro': cv_results['test_f1_macro'].mean(),
        'test_f1_macro_std': cv_results['test_f1_macro'].std(),
        'test_precision_macro': cv_results['test_precision_macro'].mean(),
        'test_recall_macro': cv_results['test_recall_macro'].mean(),
        # Écarts et overfitting
        'accuracy_gap': accuracy_gap_cv,
        'f1_gap': f1_gap_cv,
        'overfitting': overfitting_flag_cv,
        'confusion_matrix': 'N/A',
    }

    return results

# Fonction de Comparaison des Modeles

- DummyClassifier : strategy='most_frequent'
- RandomForestClassifier : n_estimators=100,max_depth=10
- XGBClassifier : eval_metric='logloss'
- CatBoostClassifier

In [128]:
def compare_models(X, y, test_size=0.2, random_state=666, cv_folds=5):
    """
    Compare tous les modèles et génère les fichiers CSV de résultats.

    Args:
        X: Features (DataFrame ou array)
        y: Labels (Series ou array)
        test_size: Proportion du jeu de test (défaut: 0.2)
        random_state: Seed pour la reproductibilité (défaut: 42)
        cv_folds: Nombre de folds pour la validation croisée (défaut: 5)

    Returns:
        tuple: (résultats_df, rapports_détaillés)
    """
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Définition des modèles
    models = {
        'DummyClassifier': DummyClassifier(
            strategy='most_frequent',
            random_state=random_state),

        'RandomForestClassifier': RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=2,
            random_state=random_state),

        'XGBClassifier': XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1,     # Plus petit = apprentissage plus lent
            subsample=0.8,          # 80% des données par arbre
            reg_lambda=2,         # Régularisation L2
            eval_metric='logloss',
            random_state=random_state),

        'CatBoostClassifier': CatBoostClassifier(
            iterations=100,
            depth=6,
            learning_rate=0.1,
            l2_leaf_reg=1.0,        # Régularisation
            verbose=False,
            random_state=random_state)
    }

    #RandomForestClassifier 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100
    #XGBClassifier          'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_lambda': 2, 'subsample': 0.8
    #CatBoostClassifier     'depth': 6, 'iterations': 100, 'l2_leaf_reg': 1, 'learning_rate': 0.1

    all_results = []
    detailed_reports = []
    all_curves = []

    print("=" * 70)
    print("COMPARAISON DES MODÈLES DE CLASSIFICATION")
    print("=" * 70)

    for model_name, model in models.items():
        print(f"\n>>> Évaluation de {model_name}...")

        # Train/Test
        print(f"  - Train/Test split...")
        tt_results, report, cm, curves = train_test_evaluation(model, X_train, X_test, y_train, y_test, model_name)
        all_results.append(tt_results)

        # Sauvegarde du rapport détaillé
        detailed_reports.append({
            'model': model_name,
            'method': 'train_test',
            'report': report,
            'confusion_matrix': cm
        })

        # Sauvegarde des courbes
        all_curves.append(curves)

        # Cross-validation
        print(f"  - Validation croisée ({cv_folds} folds)...")
        cv_results = cross_validation_evaluation(model, X, y, model_name, cv=cv_folds)
        all_results.append(cv_results)

        print(f"  ✓ {model_name} terminé")

    # Conversion en DataFrame
    results_df = pd.DataFrame(all_results)

    # Réorganisation des colonnes pour la lisibilité
    cols_order = ['model', 'method',
                  'train_accuracy', 'test_accuracy', 'accuracy_gap',
                  'train_f1_macro', 'test_f1_macro', 'f1_gap',
                  'overfitting',
                  'train_precision_macro', 'test_precision_macro',
                  'train_recall_macro', 'test_recall_macro']

    # Ajout des colonnes std si elles existent
    std_cols = [col for col in results_df.columns if '_std' in col]
    cols_order.extend(std_cols)
    cols_order.append('confusion_matrix')

    # Colonnes présentes dans le DataFrame
    cols_order = [col for col in cols_order if col in results_df.columns]
    results_df = results_df[cols_order]

    return results_df, detailed_reports, all_curves

# Fonction de Sauvegarde des Résultats
- classification_results_by_class.csv => Rapport de classification par classes
- classification_results_confusion_matrices.csv => Matrices de confusion
- classification_results_summary.csv => Tous les scores
- classification_results_overfitting_analysis.csv => Analyse spécifique de l'overfitting

In [129]:
def save_results(results_df, detailed_reports, output_prefix, curves=None):
    """
    Sauvegarde les résultats dans des fichiers CSV.

    Args:
        results_df: DataFrame avec tous les résultats
        detailed_reports: Liste des rapports détaillés
        output_prefix: Préfixe pour les fichiers de sortie
               models_data: Liste de dict avec {
            'model': nom du modèle,
            'y_true': vraies étiquettes,
            'y_pred_proba': probabilités prédites (si disponible),
            'y_pred': prédictions
        }
    """
    # 1. Fichier principal avec toutes les métriques
    results_df.to_csv(f'exports/{output_prefix}_summary.csv', index=False)
    print(f"\n✓ Résumé sauvegardé: {output_prefix}_summary.csv")

    # 2. Fichier avec les rapports détaillés de classification
    detailed_data = []
    for item in detailed_reports:
        model = item['model']
        report = item['report']

        # Extraction des métriques par classe
        for class_label, metrics in report.items():
            if class_label not in ['accuracy', 'macro avg', 'weighted avg']:
                detailed_data.append({
                    'model': model,
                    'class': class_label,
                    'precision': metrics['precision'],
                    'recall': metrics['recall'],
                    'f1-score': metrics['f1-score'],
                    'support': metrics['support']
                })

    detailed_df = pd.DataFrame(detailed_data)
    detailed_df.to_csv(f'exports/{output_prefix}_by_class.csv', index=False)
    print(f"✓ Résultats par classe sauvegardés: {output_prefix}_by_class.csv")

    # 3. Fichier avec les matrices de confusion
    cm_data = []
    for item in detailed_reports:
        cm_data.append({
            'model': item['model'],
            'confusion_matrix': str(item['confusion_matrix'].tolist())
        })

    cm_df = pd.DataFrame(cm_data)
    cm_df.to_csv(f'exports/{output_prefix}_confusion_matrices.csv', index=False)
    print(f"✓ Matrices de confusion sauvegardées: {output_prefix}_confusion_matrices.csv")

    # 4. Fichier spécifique pour l'analyse d'overfitting
    overfitting_data = results_df[['model', 'method', 'train_accuracy', 'test_accuracy',
                                     'accuracy_gap', 'train_f1_macro', 'test_f1_macro',
                                     'f1_gap', 'overfitting']].copy()
    overfitting_data.to_csv(f'{output_prefix}_overfitting_analysis.csv', index=False)
    print(f"✓ Analyse d'overfitting sauvegardée: {output_prefix}_overfitting_analysis.csv")

    # 5. Génération des graphiques ROC et Précision-Rappel
    if curves is not None:
        print("\n📈 Génération des graphiques...")
        _plot_curves(curves, output_prefix)

    print("\n" + "=" * 70)
    print("TOUS LES RÉSULTATS ONT ÉTÉ SAUVEGARDÉS")
    print("=" * 70)

    # Affichage d'un résumé de l'overfitting
    print("\n📊 RÉSUMÉ DE L'OVERFITTING:")
    print("-" * 70)
    for _, row in overfitting_data.iterrows():
        status = "⚠️  OVERFITTING DÉTECTÉ" if row['overfitting'] == 'OUI' else "✓  Pas d'overfitting"
        print(f"{row['model']:25s} ({row['method']:17s}): {status}")
        print(f"  → Écart accuracy: {row['accuracy_gap']:+.4f} | Écart F1: {row['f1_gap']:+.4f}")
    print("-" * 70)


# Fonctions de Génération de graphiques

In [130]:
def _plot_curves(models_data, output_prefix):
    """
    Génère et sauvegarde les courbes ROC pour tous les modèles.
    """
    # Déterminer le nombre de classes
    y_true_sample = models_data[0]['y_true']
    classes = np.unique(y_true_sample)
    n_classes = len(classes)

    if n_classes == 2:
        # Classification binaire
        _plot_binary_roc_curves(models_data, output_prefix)
        _plot_binary_pr_curves(models_data, output_prefix)
    else:
        # Classification multi-classes
        _plot_multiclass_roc_curves(models_data, output_prefix, classes)
        _plot_multiclass_pr_curves(models_data, output_prefix, classes)

def _plot_binary_roc_curves(models_data, output_prefix):
    """
    Courbes ROC pour classification binaire.
    """
    plt.figure(figsize=(10, 8))

    for data in models_data:
        model_name = data['model']
        y_true = data['y_true']

        # Vérifier si les probabilités sont disponibles
        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
            # Si format (n_samples, n_classes), prendre la colonne de la classe positive
            if len(y_score.shape) > 1:
                y_score = y_score[:, 1]
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, utilisation des prédictions")
            y_score = data['y_pred']

        # Calculer la courbe ROC
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f'{model_name} (AUC = {roc_auc:.3f})')

    # Ligne diagonale (classificateur aléatoire)
    plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Aléatoire (AUC = 0.500)')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Taux de Faux Positifs', fontsize=12)
    plt.ylabel('Taux de Vrais Positifs', fontsize=12)
    plt.title('Courbes ROC - Comparaison des Modèles', fontsize=14, fontweight='bold')
    plt.legend(loc="lower right", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    filename = f'images/{output_prefix}_{model_name}_roc_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes ROC sauvegardées: {output_prefix}_{model_name}_roc_curves.png")

def _plot_multiclass_roc_curves(models_data, output_prefix, classes):
    """
    Courbes ROC pour classification multi-classes (One-vs-Rest).
    """
    n_classes = len(classes)
    n_models = len(models_data)

    # Créer une figure avec sous-graphiques
    fig, axes = plt.subplots(1, n_models, figsize=(8 * n_models, 6))
    if n_models == 1:
        axes = [axes]

    for idx, data in enumerate(models_data):
        ax = axes[idx]
        model_name = data['model']
        y_true = data['y_true']

        # Binariser les labels
        y_true_bin = label_binarize(y_true, classes=classes)

        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, graphique ROC limité")
            continue

        # Calculer ROC pour chaque classe
        fpr = dict()
        tpr = dict()
        roc_auc = dict()

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
            ax.plot(fpr[i], tpr[i], lw=2,
                    label=f'Classe {classes[i]} (AUC = {roc_auc[i]:.3f})')

        # Calculer micro-average ROC
        fpr_micro, tpr_micro, _ = roc_curve(y_true_bin.ravel(), y_score.ravel())
        roc_auc_micro = auc(fpr_micro, tpr_micro)
        ax.plot(fpr_micro, tpr_micro, lw=3, linestyle='--',
                label=f'Micro-avg (AUC = {roc_auc_micro:.3f})', color='navy')

        ax.plot([0, 1], [0, 1], 'k--', lw=2)
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('Taux de Faux Positifs', fontsize=11)
        ax.set_ylabel('Taux de Vrais Positifs', fontsize=11)
        ax.set_title(f'ROC - {model_name}', fontsize=12, fontweight='bold')
        ax.legend(loc="lower right", fontsize=9)
        ax.grid(alpha=0.3)

    plt.tight_layout()
    filename = f'images/{output_prefix}_{model_name}_roc_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes ROC sauvegardées: {output_prefix}_{model_name}_roc_curves.png")

def _plot_binary_pr_curves(models_data, output_prefix):
    """
    Courbes Précision-Rappel pour classification binaire.
    """
    plt.figure(figsize=(10, 8))

    for data in models_data:
        model_name = data['model']
        y_true = data['y_true']

        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
            if len(y_score.shape) > 1:
                y_score = y_score[:, 1]
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, utilisation des prédictions")
            y_score = data['y_pred']

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        avg_precision = average_precision_score(y_true, y_score)

        plt.plot(recall, precision, lw=2,
                 label=f'{model_name} (AP = {avg_precision:.3f})')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Rappel', fontsize=12)
    plt.ylabel('Précision', fontsize=12)
    plt.title('Courbes Précision-Rappel - Comparaison des Modèles',
              fontsize=14, fontweight='bold')
    plt.legend(loc="lower left", fontsize=10)
    plt.grid(alpha=0.3)
    plt.tight_layout()

    filename = f'images/{output_prefix}_{model_name}_precision_recall_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes Précision-Rappel sauvegardées: {output_prefix}_{model_name}_precision_recall_curves.png")

def _plot_multiclass_pr_curves(models_data, output_prefix, classes):
    """
    Courbes Précision-Rappel pour classification multi-classes.
    """
    n_classes = len(classes)
    n_models = len(models_data)

    fig, axes = plt.subplots(1, n_models, figsize=(8 * n_models, 6))
    if n_models == 1:
        axes = [axes]

    for idx, data in enumerate(models_data):
        ax = axes[idx]
        model_name = data['model']
        y_true = data['y_true']

        y_true_bin = label_binarize(y_true, classes=classes)

        if 'y_pred_proba' in data and data['y_pred_proba'] is not None:
            y_score = data['y_pred_proba']
        else:
            print(f"⚠️  Pas de probabilités pour {model_name}, graphique PR limité")
            continue

        # Calculer PR pour chaque classe
        for i in range(n_classes):
            precision, recall, _ = precision_recall_curve(y_true_bin[:, i],
                                                          y_score[:, i])
            avg_precision = average_precision_score(y_true_bin[:, i], y_score[:, i])
            ax.plot(recall, precision, lw=2,
                    label=f'Classe {classes[i]} (AP = {avg_precision:.3f})')

        # Micro-average
        precision_micro, recall_micro, _ = precision_recall_curve(
            y_true_bin.ravel(), y_score.ravel())
        avg_precision_micro = average_precision_score(y_true_bin, y_score,
                                                      average='micro')
        ax.plot(recall_micro, precision_micro, lw=3, linestyle='--',
                label=f'Micro-avg (AP = {avg_precision_micro:.3f})', color='navy')

        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('Rappel', fontsize=11)
        ax.set_ylabel('Précision', fontsize=11)
        ax.set_title(f'Précision-Rappel - {model_name}', fontsize=12, fontweight='bold')
        ax.legend(loc="lower left", fontsize=9)
        ax.grid(alpha=0.3)

    plt.tight_layout()
    filename = f'images/{output_prefix}_{model_name}_precision_recall_curves.png'
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Courbes Précision-Rappel sauvegardées: {output_prefix}_{model_name}_precision_recall_curves.png")

# Affichage des Résultats

In [131]:
# Comparaison des modèles
results_df, detailed_reports, all_curves = compare_models(X, y, test_size=0.15, cv_folds=5)

# Sauvegarde des résultats
save_results(results_df, detailed_reports, output_prefix='classification_results', curves=all_curves)

COMPARAISON DES MODÈLES DE CLASSIFICATION

>>> Évaluation de DummyClassifier...
  - Train/Test split...
  - Validation croisée (5 folds)...
  ✓ DummyClassifier terminé

>>> Évaluation de RandomForestClassifier...
  - Train/Test split...
  - Validation croisée (5 folds)...


/Users/tico/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tico/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tico/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/

  ✓ RandomForestClassifier terminé

>>> Évaluation de XGBClassifier...
  - Train/Test split...
  - Validation croisée (5 folds)...
  ✓ XGBClassifier terminé

>>> Évaluation de CatBoostClassifier...
  - Train/Test split...
  - Validation croisée (5 folds)...
  ✓ CatBoostClassifier terminé

✓ Résumé sauvegardé: classification_results_summary.csv
✓ Résultats par classe sauvegardés: classification_results_by_class.csv
✓ Matrices de confusion sauvegardées: classification_results_confusion_matrices.csv
✓ Analyse d'overfitting sauvegardée: classification_results_overfitting_analysis.csv

📈 Génération des graphiques...
✓ Courbes ROC sauvegardées: classification_results_CatBoostClassifier_roc_curves.png
✓ Courbes Précision-Rappel sauvegardées: classification_results_CatBoostClassifier_precision_recall_curves.png

TOUS LES RÉSULTATS ONT ÉTÉ SAUVEGARDÉS

📊 RÉSUMÉ DE L'OVERFITTING:
----------------------------------------------------------------------
DummyClassifier           (train_test       ):

# Modele DUMMY
- DummyClassifier

 # Modele LINEAIRE

# Modele NON LINEAIRE
- RandomForest, XGBoost ou CatBoost
- Métriques d’évaluation en classification : matrice de confusion, rappel et précision.
- Scores (présence d’overfit ou non, capacité d’éviter les faux positifs ou faux négatifs)

# Amélioration de la classification
- demandez-vous si éviter des faux positifs est plus important qu’éviter des faux négatifs.

# OPTIMISATION DES HYPER-PARAMETRES

In [132]:
def get_param_grids():
    """
    Définit les grilles de paramètres pour chaque modèle.

    Returns:
        dict: Dictionnaire avec les grilles de paramètres pour chaque modèle
    """
    param_grids = {
        'DummyClassifier': {
            'strategy': ['most_frequent', 'stratified', 'uniform']
        },

        'RandomForestClassifier': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3,4,5,6,7,8,9, None],
            'min_samples_split': [2, 5, 10, 15, 20],
            'min_samples_leaf': [1, 5, 10],
            'max_features': ['sqrt', 'log2']
        },

        'XGBClassifier': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'reg_alpha': [0, 0.1, 1],
            'reg_lambda': [1, 2, 5]
        },

        'CatBoostClassifier': {
            'iterations': [100, 200, 300],
            'depth': [4, 6, 8],
            'learning_rate': [0.01, 0.05, 0.1],
            'l2_leaf_reg': [1, 3, 5],
            'subsample': [0.8, 1.0]
        }
    }

    return param_grids

def get_small_param_grids():
    """
    Grilles réduites pour des tests rapides.

    Returns:
        dict: Grilles de paramètres réduites
    """
    param_grids = {
        'DummyClassifier': {
            'strategy': ['most_frequent', 'stratified']
        },

        'RandomForestClassifier': {
            'n_estimators': [50, 100],
            'max_depth': [5, 10, None],
            'min_samples_split': [2, 10],
            'max_features': ['sqrt']
        },

        'XGBClassifier': {
            'n_estimators': [50, 100],
            'max_depth': [3, 5],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0],
            'reg_lambda': [1, 2]
        },

        'CatBoostClassifier': {
            'iterations': [50, 100],
            'depth': [4, 6],
            'learning_rate': [0.01, 0.1],
            'l2_leaf_reg': [1, 3]
        }
    }

    return param_grids

def perform_grid_search(model, param_grid, X_train, y_train, model_name,
                       cv=5, scoring='f1_macro', n_jobs=-1):
    """
    Effectue un GridSearchCV pour un modèle donné.

    Args:
        model: Le modèle à optimiser
        param_grid: Grille de paramètres
        X_train, y_train: Données d'entraînement
        model_name: Nom du modèle
        cv: Nombre de folds pour la validation croisée
        scoring: Métrique d'optimisation
        n_jobs: Nombre de processus parallèles

    Returns:
        tuple: (GridSearchCV object, résultats dict)
    """
    print(f"\n{'='*70}")
    print(f"🔍 Grid Search pour {model_name}")
    print(f"{'='*70}")
    print(f"Nombre de combinaisons à tester: {np.prod([len(v) for v in param_grid.values()])}")
    print(f"Métrique d'optimisation: {scoring}")
    print(f"Validation croisée: {cv} folds")

    start_time = time.time()

    # Configuration de la validation croisée stratifiée
    cv_strategy = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    # GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv_strategy,
        scoring=scoring,
        n_jobs=n_jobs,
        verbose=0,
        return_train_score=True
    )

    # Entraînement
    grid_search.fit(X_train, y_train)

    elapsed_time = time.time() - start_time

    # Résultats
    results = {
        'model': model_name,
        'best_score': grid_search.best_score_,
        'best_params': str(grid_search.best_params_),
        'n_combinations': len(grid_search.cv_results_['params']),
        'time_seconds': elapsed_time
    }

    print(f"✓ Terminé en {elapsed_time:.2f} secondes")
    print(f"📊 Meilleur score ({scoring}): {grid_search.best_score_:.4f}")
    print(f"🏆 Meilleurs paramètres:")
    for param, value in grid_search.best_params_.items():
        print(f"   - {param}: {value}")

    return grid_search, results

def evaluate_best_model(grid_search, X_train, X_test, y_train, y_test, model_name):
    """
    Évalue le meilleur modèle trouvé par GridSearch sur le jeu de test.

    Args:
        grid_search: Objet GridSearchCV entraîné
        X_train, X_test: Features
        y_train, y_test: Labels
        model_name: Nom du modèle

    Returns:
        dict: Résultats d'évaluation
    """
    best_model = grid_search.best_estimator_

    # Prédictions
    y_pred_train = best_model.predict(X_train)
    y_pred_test = best_model.predict(X_test)

    # Rapports
    report_train = classification_report(y_train, y_pred_train, output_dict=True)
    report_test = classification_report(y_test, y_pred_test, output_dict=True)

    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred_test)

    # Calcul des écarts (overfitting)
    accuracy_gap = report_train['accuracy'] - report_test['accuracy']
    f1_gap = report_train['macro avg']['f1-score'] - report_test['macro avg']['f1-score']
    overfitting = 'OUI' if (accuracy_gap > 0.12 or f1_gap > 0.12) else 'NON'

    results = {
        'model': model_name,
        'train_accuracy': report_train['accuracy'],
        'test_accuracy': report_test['accuracy'],
        'train_f1_macro': report_train['macro avg']['f1-score'],
        'test_f1_macro': report_test['macro avg']['f1-score'],
        'train_precision': report_train['macro avg']['precision'],
        'test_precision': report_test['macro avg']['precision'],
        'train_recall': report_train['macro avg']['recall'],
        'test_recall': report_test['macro avg']['recall'],
        'accuracy_gap': accuracy_gap,
        'f1_gap': f1_gap,
        'overfitting': overfitting,
        'confusion_matrix': str(cm.tolist())
    }

    print(f"\n📈 Évaluation sur le jeu de test:")
    print(f"   Train accuracy: {report_train['accuracy']:.4f}")
    print(f"   Test accuracy:  {report_test['accuracy']:.4f}")
    print(f"   Écart accuracy: {accuracy_gap:+.4f}")
    print(f"   Overfitting:    {overfitting}")

    return results, report_test, cm

def compare_models_gridsearch(X, y, param_grids='full', test_size=0.15,
                              cv=5, scoring='f1_macro', random_state=666):
    """
    Compare tous les modèles avec GridSearchCV.

    Args:
        X: Features
        y: Labels
        param_grids: 'full', 'small', ou dict personnalisé
        test_size: Proportion du jeu de test
        cv: Nombre de folds
        scoring: Métrique d'optimisation
        random_state: Seed

    Returns:
        tuple: (résultats_grid, résultats_eval, grid_objects)
    """
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    print(f"\n{'='*70}")
    print(f"GRIDSEARCH - COMPARAISON DES MODÈLES")
    print(f"{'='*70}")
    print(f"Taille du dataset: {len(X)} échantillons")
    print(f"Train: {len(X_train)} | Test: {len(X_test)}")
    print(f"Distribution des classes: {dict(pd.Series(y).value_counts())}")

    # Sélection de la grille
    if param_grids == 'full':
        grids = get_param_grids()
    elif param_grids == 'small':
        grids = get_small_param_grids()
    else:
        grids = param_grids

    # Modèles de base
    base_models = {
        'DummyClassifier': DummyClassifier(random_state=random_state),
        'RandomForestClassifier': RandomForestClassifier(random_state=random_state),
        'XGBClassifier': XGBClassifier(random_state=random_state, eval_metric='logloss'),
        'CatBoostClassifier': CatBoostClassifier(random_state=random_state, verbose=False)
    }

    grid_results = []
    eval_results = []
    grid_objects = {}
    detailed_reports = []

    # GridSearch pour chaque modèle
    for model_name, base_model in base_models.items():
        param_grid = grids[model_name]

        # GridSearch
        grid_search, grid_res = perform_grid_search(
            base_model, param_grid, X_train, y_train,
            model_name, cv=cv, scoring=scoring
        )
        grid_results.append(grid_res)
        grid_objects[model_name] = grid_search

        # Évaluation du meilleur modèle
        eval_res, report, cm = evaluate_best_model(
            grid_search, X_train, X_test, y_train, y_test, model_name
        )
        eval_results.append(eval_res)

        detailed_reports.append({
            'model': model_name,
            'report': report,
            'confusion_matrix': cm
        })

    # Conversion en DataFrames
    grid_df = pd.DataFrame(grid_results)
    eval_df = pd.DataFrame(eval_results)

    return grid_df, eval_df, grid_objects, detailed_reports

def save_gridsearch_results(grid_df, eval_df, grid_objects, detailed_reports,
                            output_prefix='gridsearch_results'):
    """
    Sauvegarde tous les résultats du GridSearch.

    Args:
        grid_df: DataFrame avec résultats du GridSearch
        eval_df: DataFrame avec évaluation finale
        grid_objects: Dict des objets GridSearchCV
        detailed_reports: Rapports détaillés
        output_prefix: Préfixe des fichiers
    """
    # 1. Résumé du GridSearch
    grid_df.to_csv(f'exports/{output_prefix}_grid_summary.csv', index=False)
    print(f"\n✓ Résumé GridSearch: {output_prefix}_grid_summary.csv")

    # 2. Évaluation finale
    eval_df.to_csv(f'exports/{output_prefix}_evaluation.csv', index=False)
    print(f"✓ Évaluation finale: {output_prefix}_evaluation.csv")

    # 3. Détails complets du GridSearch (tous les résultats)
    all_cv_results = []
    for model_name, grid_search in grid_objects.items():
        cv_res = pd.DataFrame(grid_search.cv_results_)
        cv_res.insert(0, 'model', model_name)
        all_cv_results.append(cv_res)

    full_cv_df = pd.concat(all_cv_results, ignore_index=True)
    full_cv_df.to_csv(f'exports/{output_prefix}_full_cv_results.csv', index=False)
    print(f"✓ Résultats CV complets: {output_prefix}_full_cv_results.csv")

    # 4. Meilleurs paramètres
    best_params_data = []
    for model_name, grid_search in grid_objects.items():
        for param, value in grid_search.best_params_.items():
            best_params_data.append({
                'model': model_name,
                'parameter': param,
                'value': str(value)
            })

    best_params_df = pd.DataFrame(best_params_data)
    best_params_df.to_csv(f'exports/{output_prefix}_best_params.csv', index=False)
    print(f"✓ Meilleurs paramètres: {output_prefix}_best_params.csv")

    # 5. Matrices de confusion
    cm_data = []
    for item in detailed_reports:
        cm_data.append({
            'model': item['model'],
            'confusion_matrix': str(item['confusion_matrix'].tolist())
        })
    cm_df = pd.DataFrame(cm_data)
    cm_df.to_csv(f'exports/{output_prefix}_confusion_matrices.csv', index=False)
    print(f"✓ Matrices de confusion: {output_prefix}_confusion_matrices.csv")

    # 6. Analyse d'overfitting
    overfitting_cols = ['model', 'train_accuracy', 'test_accuracy', 'accuracy_gap',
                       'train_f1_macro', 'test_f1_macro', 'f1_gap', 'overfitting']
    overfitting_df = eval_df[overfitting_cols]
    overfitting_df.to_csv(f'{output_prefix}_overfitting.csv', index=False)
    print(f"✓ Analyse overfitting: {output_prefix}_overfitting.csv")

    print(f"\n{'='*70}")
    print("TOUS LES RÉSULTATS ONT ÉTÉ SAUVEGARDÉS")
    print(f"{'='*70}")

    # Affichage du classement
    print(f"\n🏆 CLASSEMENT DES MODÈLES (par {grid_objects[list(grid_objects.keys())[0]].scoring}):")
    print("-" * 70)
    ranking = grid_df.sort_values('best_score', ascending=False)
    for i, row in ranking.iterrows():
        print(f"{i+1}. {row['model']:25s} - Score: {row['best_score']:.4f} - Temps: {row['time_seconds']:.1f}s")

    print("\n📊 DÉTECTION D'OVERFITTING:")
    print("-" * 70)
    for _, row in eval_df.iterrows():
        status = "⚠️  OVERFITTING" if row['overfitting'] == 'OUI' else "✓  Pas d'overfitting"
        print(f"{row['model']:25s}: {status} (écart: {row['accuracy_gap']:+.4f})")

# GridSearch avec grille réduite (rapide pour test)
# Utilisez param_grids='full' pour une recherche complète
grid_df, eval_df, grid_objects, detailed_reports = compare_models_gridsearch(
    X, y,
    param_grids='full',  # 'small', 'full', ou dict personnalisé
    test_size=0.15,
    cv=5,  # 3 pour test rapide, 5 recommandé
    scoring='f1_macro',
    random_state=666
)

# Sauvegarde
save_gridsearch_results(
    grid_df, eval_df, grid_objects, detailed_reports,
    output_prefix='gridsearch_results'
)

print("\n" + "="*70)
print("APERÇU DES MEILLEURS RÉSULTATS")
print("="*70)
print(grid_df.round(4).to_string(index=False))


GRIDSEARCH - COMPARAISON DES MODÈLES
Taille du dataset: 1470 échantillons
Train: 1249 | Test: 221
Distribution des classes: {False: np.int64(1233), True: np.int64(237)}

🔍 Grid Search pour DummyClassifier
Nombre de combinaisons à tester: 3
Métrique d'optimisation: f1_macro
Validation croisée: 5 folds
✓ Terminé en 1.52 secondes
📊 Meilleur score (f1_macro): 0.4877
🏆 Meilleurs paramètres:
   - strategy: stratified

📈 Évaluation sur le jeu de test:
   Train accuracy: 0.7278
   Test accuracy:  0.7330
   Écart accuracy: -0.0052
   Overfitting:    NON

🔍 Grid Search pour RandomForestClassifier
Nombre de combinaisons à tester: 720
Métrique d'optimisation: f1_macro
Validation croisée: 5 folds


KeyboardInterrupt: 